In [25]:
%%writefile in.langevin
# Sample LAMMPS input script for thermal conductivity of liquid LJ: thermostatting 2 regions via fix langevin

# SETTINGS
variable        x equal 10
variable        y equal 10
variable        z equal 20

variable        rho equal 0.6   # Reduced density. For fcc unit cell, the formula appears to be rho*=4/V^3. Sigma may be just 1.
variable        rc equal 2.5
variable        tlo equal 1.0
variable        thi equal 1.70
variable        t equal 1.35    # Midway between tlo and thi


# PROBLEM SETUP
units           lj
atom_style      atomic

lattice         fcc ${rho}
region          box block 0 $x 0 $y 0 $z
create_box      1 box
create_atoms    1 box
mass            1 1.0

velocity        all create $t 87287

pair_style      lj/cut ${rc}
pair_coeff      1 1 1.0 1.0     # epsilon, sigma

# Heat layers
region          hot block INF INF INF INF 0 1
region          cold block  INF INF INF INF 10 11
compute         Thot all temp/region hot
compute         Tcold all temp/region cold


# EQUILIBRATION 1
# Equilibrate the system with a single temperature, t
fix             1 all nvt temp $t $t 0.5
thermo          100

shell if [ -d equil_1 ]; then rm -rf equil_1; fi
shell mkdir equil_1
dump            1 all custom 100 equil_1/dump.equil_*.cfg id mass type x y z
run             5000

velocity        all scale $t

unfix           1
undump          1

# EQUILIBRATION 2
# Equilibrate the hot and cold regions
reset_timestep  0

fix             1 all nve

# tally will keep track of energy changes in the hot and cold regions
fix             hot all langevin ${thi} ${thi} 1.0 59804 tally yes
fix_modify      hot temp Thot

fix             cold all langevin ${tlo} ${tlo} 1.0 287859 tally yes
fix_modify      cold temp Tcold

variable        step equal step
variable        tdiff equal c_Thot-c_Tcold

thermo_style    custom step temp c_Thot c_Tcold f_hot f_cold v_tdiff
thermo_modify   colname c_Thot Temp_hot colname c_Tcold Temp_cold &
                colname f_hot E_hot colname f_cold E_cold &
                colname v_tdiff dTemp_step
thermo          1000

shell if [ -d equil_2 ]; then rm -rf equil_2; fi
shell mkdir equil_2
dump            2 all custom 100 equil_2/dump.equil_*.cfg id mass type x y z

run             5000

undump          2

# THERMAL CONDUCTIVITY 
# Collect information following the previous equilibration
reset_timestep  0

# Every atom has an average kinetic energy of ⁠3/2 kB T in thermal equilibrium. In lj units, kB = 1
compute         ke all ke/atom
variable        temp atom c_ke/1.5

fix             1 all nve

# Reset langevin thermostats to zero energy accumulation
fix             hot all langevin ${thi} ${thi} 1.0 59804 tally yes
fix_modify      hot temp Thot

fix             cold all langevin ${tlo} ${tlo} 1.0 287859 tally yes
fix_modify      cold temp Tcold

# Calculates the running average of the temperature difference between the hot and cold regions
# Input values every 10 steps. Running average is updated every 1000 steps, updated using the latest 100 samples.
fix             print_tdiff all print 10 "${step} ${tdiff}" file tdiff_values.dat screen no
fix             ave all ave/time 10 100 1000 v_tdiff ave running file ave_tdiff.dat   

thermo          1000
thermo_style    custom step temp c_Thot c_Tcold f_hot f_cold v_tdiff f_ave
thermo_modify   colname c_Thot Temp_hot colname c_Tcold Temp_cold &
                colname f_hot E_hot colname f_cold E_cold &
                colname v_tdiff dTemp_step colname f_ave dTemp_ave

# Bin the atoms into 20 layers along the z-axis
compute         layers all chunk/atom bin/1d z lower 0.05 units reduced

# Block average - Input values every 10 steps (average v_temp in each layer)
# In each layer, calculate the average temperature every 1000 steps, using the latest 100 samples.
# Coord1 in profile.langevin represents the center point of the bin
#fix             test all ave/chunk 100 1 100 layers v_temp file profile.langevin_test
fix             2 all ave/chunk 100 10 1000 layers v_temp file profile.langevin

# Continue here!
variable start_time equal time

# f_hot and f_cold are the cumulative energies added/removed in the hot and cold regions, respectively.
# 0.5 * (abs(f_hot) + abs(f_cold)) is delta Q. The factor of 0.5 is because the total heat flux is divided equally between the hot and cold regions.
# lz/2.0 is the distance from the middle of the hot region to the middle of the cold region.
variable kappa equal (0.5*(abs(f_hot)+abs(f_cold))/(time-${start_time})/(lx*ly)/2.0)*(lz/2.0)/f_ave

shell if [ -d kappa ]; then rm -rf kappa; fi
shell mkdir kappa
dump            3 all custom 100 kappa/dump.kappa_*.cfg id mass type x y z v_temp

run             5000
print           "Running average thermal conductivity: $(v_kappa:%.2f)"


Overwriting in.langevin


In [26]:
!lmp < in.langevin

LAMMPS (29 Aug 2024 - Development - 2995cb76ae)
OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (src/comm.cpp:98)
  using 1 OpenMP thread(s) per MPI task
Lattice spacing in x,y,z = 1.8820721 1.8820721 1.8820721
Created orthogonal box = (0 0 0) to (18.820721 18.820721 37.641441)
  1 by 1 by 1 MPI processor grid
Created 8000 atoms
  using lattice units in orthogonal box = (0 0 0) to (18.820721 18.820721 37.641441)
  create_atoms CPU = 0.002 seconds
Generated 0 of 0 mixed pair_coeff terms from geometric mixing rule
Neighbor list info ...
  update: every = 1 steps, delay = 0 steps, check = yes
  max neighbors/atom: 2000, page size: 100000
  master list distance cutoff = 2.8
  ghost atom cutoff = 2.8
  binsize = 1.4, bins = 14 14 27
  1 neighbor lists, perpetual/occasional/extra = 1 0 0
  (1) pair lj/cut, perpetual
      attributes: half, newton on
      pair build: half/bin/atomonly/newton
      stencil: half/bin/3d
      bin: standard
Setting up Verlet run ...
  Unit style